<a href="https://colab.research.google.com/github/Spnetic-5/clarifAI/blob/main/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re
from keras.utils import to_categorical
import nltk
from nltk.tokenize import word_tokenize
from google.colab import files
nltk.download('punkt')
from google.colab import drive
drive.mount("/content/gdrive")
# uploaded = files.upload()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:

data_path = '/content/gdrive/My Drive/merged-file.txt'
with open(data_path, 'r') as f:
    lines = f.read().split('\n')

In [3]:
# print(len(lines))
data = " ".join(lines)

cleaned = data.lower()

tokens = word_tokenize(cleaned)
train_len = 3+1
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)
sequences = {}
count = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]


In [ ]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
train_inputs.shape
#print(train_targets[0])

In [ ]:
train_targets[0]

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

#LSTM Model
model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len)) # Embedding Layer
model.add(LSTM(50,return_sequences=True))  # 1st LSTM layer with 50 hidden layers
model.add(Dropout(0.2))  # prevent a model from overfitting
model.add(LSTM(50))  # 2nd LSTM layer with 50 hidden layers
model.add(Dense(50,activation='relu'))  #Dense layer with 50 neurons and relu activation
model.add(Dense(vocabulary_size, activation='softmax')) #Probability for each predicted word 
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_inputs,train_targets,epochs=500,verbose=1)
model.save("mymodel.h5")

In [ ]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print(encoded_text, pad_encoded)
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)